In [61]:
import chromadb
from groq import Groq
from dotenv import load_dotenv 
import os

In [62]:
# Instance the APP VARIABLES
SYSTEM_PROMPT_PATH = "../system_prompt.txt"
CHROMA_DB = "../chromadb/persistent_db/"

In [63]:
chroma_client = chromadb.PersistentClient(path=CHROMA_DB)
collection = chroma_client.get_or_create_collection(
    name="cv_doc",
    metadata = {"hnsw:space": "cosine"}
)

In [64]:
class Chatbot:

    # Inits the basic atributes of the chatbot
    def __init__(self, system_prompt_path: str):
        load_dotenv()
        self.api_key = os.getenv("GROQ_API_KEY")
        self.system_prompt = self.read_doc(system_prompt_path) # init system_prompt
        self.context = self.system_prompt #init context
        self.documents = "" #init documents where data is extracted
        self.messages = [{"role": "system","content" : self.context}] #init of messages of the chat
        self.client = Groq(api_key= self.api_key)

  
    # Gets the user input message
    def get_user_message(self):
        return input("Escribe tu mensaje: ")
    
    # Appends user message in the message history
    def add_user_message(self, message: str):
        self.messages.append({"role": "user", "content": message})
    
    # Add the role assistant to the message pool list.
    def add_assistant_message(self, message: str):
        self.messages.append({"role": "assistant", "content": message})

    # Reads the documents from the system
    def read_doc(self, path: str) -> str:
        with open(path, 'r') as f:
            return f.read()

    # Generates the model, params, and options    
    def get_completion(self):
        return self.client.chat.completions.create( 
            model="llama-3.1-8b-instant", 
            messages=self.messages, 
            temperature=1, 
            max_tokens=1024, 
            top_p=1, 
            stream=True,
            stop=None
        )

    # Gets the user message > gets the context documents > append the user message to chat > updates system content
    def handle_user_query(self, user_message : str) :
        self.documents = self.get_context_query(user_message) # gets the new context
        self.context = self.system_prompt + self.documents # updates the new context for the model
        self.add_user_message(user_message) # adds the user message to the chat
        self.messages[0]['content'] = self.context # update the context message with the new context
    
    #Gets the embeddings context from user message
    def get_context_query(self, user_message : str) -> str :
        query_results = collection.query(
        query_texts = user_message,
        n_results= 3
        )
        context_entries = []  # Lista para almacenar las entradas del contexto

        # Recorre los documentos y metadatos obtenidos
        for doc, metadata in zip(query_results['documents'][0], query_results['metadatas'][0]):
            # Formatear el texto del documento y los metadatos
            context_entry = f'{{"data": "{doc}", "url_source": "{metadata.get("document", "N/A")}"}}'
            context_entries.append(context_entry)

        # Devuelve las entradas de contexto como una cadena
        print(context_entries)
        return str(context_entries)
        
    # Generates a iteration on the chat: add user and assistant messages to the history.
    def chat_iteration(self, user_message : str):
        self.handle_user_query(user_message)
        
        completion = self.get_completion()
        assistant_response = ""
        for chunk in completion:
            response_part = chunk.choices[0].delta.content or ""
            print(response_part, end="")
            assistant_response += response_part
        self.add_assistant_message(assistant_response)

    # Generates the chat funcion that allow users send multiple messages on a conversation
    def chat(self):
        while True:
            user_message = self.get_user_message()
            if user_message.strip().lower() == "/exit" :
                print("Hasta luego!")
                break
            self.chat_iteration(user_message) 


In [65]:
chatbot = Chatbot(SYSTEM_PROMPT_PATH)

In [ ]:
chatbot.chat()

Escribe tu mensaje:  Que estudios tiene Martín?


['{"data": "Encantado, mi nombre es Martín y soy un desarrollador de Software con 2 años de experiencia en el sector de la tecnología", "url_source": "N/A"}', '{"data": "Proyecto sobre el teorema del mono infinito que habla que un mono con tiempo infinito en algun momento escribirá shakespeare", "url_source": "CV"}', '{"data": "Mi último puesto de trabajo ha sido en el CESGA (Centro de supercomputación de Galicia, en el que he creado un chatbot)", "url_source": "CV"}']
Lo siento, pero la información proporcionada no incluye detalles específicos sobre los estudios de Martín Gago. Sin embargo, puedo hacer una suposición basada en la información proporcionada. Se menciona que Martín es un desarrollador de software con 2 años de experiencia en el sector de la tecnología, lo que sugiere que probablemente ha estudiado una carrera relacionada con la informática o la ingeniería en informática.

 ¿Hay cualquier otra pregunta en la que pueda ayudarte?car su CV o perfiles en redes profesionales c

Escribe tu mensaje:  Que proyectos ha desarrollado?


['{"data": "En el 2024 he compleado y conseguido mis titulos en Desarrollo aplicaciones multiplataforma", "url_source": "CV"}', '{"data": "Mis últimos proyectos han sido: desarrollo de un chatbot empleando inteligencia artificial, un proyecto sobre el teorema del mono infinito, una aplicación de ciudadanía móvil", "url_source": "CV"}', '{"data": "Encantado, mi nombre es Martín y soy un desarrollador de Software con 2 años de experiencia en el sector de la tecnología", "url_source": "N/A"}']
Martín Gago ha desarrollado varios proyectos interesantes, según se menciona en su CV. A continuación, te presento algunos de los proyectos en los que ha trabajado:

1. **Desarrollo de un chatbot empleando inteligencia artificial**: Martín ha desarrollado un chatbot que utiliza inteligencia artificial para interactuar con usuarios y proporcionar respuestas personalizadas.
2. **Proyecto sobre el teorema del mono infinito**: Este proyecto es un poco más abstracto, pero parece estar relacionado con una

Escribe tu mensaje:  cual es el linkedin de martin?


['{"data": "Encantado, mi nombre es Martín y soy un desarrollador de Software con 2 años de experiencia en el sector de la tecnología", "url_source": "N/A"}', '{"data": "Mi último puesto de trabajo ha sido en el CESGA (Centro de supercomputación de Galicia, en el que he creado un chatbot)", "url_source": "CV"}', '{"data": "En el 2024 he compleado y conseguido mis titulos en Desarrollo aplicaciones multiplataforma", "url_source": "CV"}']
Lamentablemente, no tengo acceso a la información de contactos personales de Martín Gago, incluyendo su perfil de LinkedIn. Sin embargo, puedo sugerir algunas opciones para que puedas contactarlo o encontrar su perfil de LinkedIn:

1. **Buscar en LinkedIn**: Puedes buscar su nombre y apellido en LinkedIn para ver si tiene un perfil activo.
2. **Buscar en redes profesionales**: Puedes buscar su nombre y apellido en otras redes profesionales, como DEV.to, GitHub, CodePen, etc.
 su enlace de perfil de LinkedIn.reo electrónico**: Si tienes acceso a su corre